In [186]:
from datetime import datetime, timedelta
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload


class GoogleAPI:
    def __init__(self):
        # Obtain credentials
        self.creds = self.get_credentials()

        # Create YouTube and Drive services
        self.youtube = self.create_youtube_service()
        self.drive = self.create_drive_service()
        self.docs = self.create_docs_service()

        # Specify IDs
        self.folder_id = '1FKQa5QuHeDidQMLIlMqad77KLCmr9WOG'
        self.channel_id = 'UCgT5Qz-GdmPf5aVnpVoQyFA'
        self.playlist_id = 'UUgT5Qz-GdmPf5aVnpVoQyFA'

    def get_credentials(self):
        # Scopes for YouTube and Google Drive
        scopes = [
            "https://www.googleapis.com/auth/documents",
            "https://www.googleapis.com/auth/drive.file",
            "https://www.googleapis.com/auth/drive",
            'https://www.googleapis.com/auth/youtube',
            'https://www.googleapis.com/auth/youtube.readonly',
            'https://www.googleapis.com/auth/youtube.upload',
            'https://www.googleapis.com/auth/youtube.force-ssl'
            ]
        
        # Create a flow instance to manage the OAuth 2.0 Authorization Grant Flow steps
        flow = InstalledAppFlow.from_client_secrets_file('API/client_secret.json', scopes=scopes)
        
        # Run the flow to obtain the credentials
        credentials = flow.run_local_server(port=50405)
        
        return credentials

    def create_youtube_service(self):
        # Build the YouTube service object
        return build('youtube', 'v3', credentials=self.creds)

    def create_drive_service(self):
        # Build the Google Drive service object
        return build('drive', 'v3', credentials=self.creds)

    def create_docs_service(self):
        # Build the Google Docs service object
        return build('docs', 'v1', credentials=self.creds)

    def list_docs(self, folder_id: str = ""):
        if not folder_id:
            folder_id = self.folder_id
        # List all Google Docs files
        results = google.drive.files().list(
                q=f"mimeType='application/vnd.google-apps.document' and trashed=false and '{folder_id}' in parents",
                fields="files(id, name)"
            ).execute()
        
        # Convert list of files to a dictionary with name as key and id as value
        docs_dict = {doc['name']: doc['id'] for doc in results.get('files', [])}

        return docs_dict
    
    def create_docs(self, name: str):
        # Define the file metadata including the name and the parent folder ID
        document_metadata = {
            'name': name,
            'mimeType': 'application/vnd.google-apps.document',
            'parents': [self.folder_id]
        }

        # Create the document using the Drive service
        document = self.drive.files().create(body=document_metadata).execute()

        print("Success: New Docs Created")

        # Return the document ID and URL for further operations or verification
        return document.get('id')

    def get_document_content(self, document_id: str):
        # Retrieve the content of a Google Docs document
        doc = self.docs.documents().get(documentId=document_id).execute()
        # Extract text content from the document
        text_content = ''
        for element in doc.get('body', {}).get('content', []):
            if 'paragraph' in element:
                for para_element in element['paragraph']['elements']:
                    if 'textRun' in para_element and 'content' in para_element['textRun']:
                        text_content += para_element['textRun']['content']
        return text_content
    
    def move_to_trash(self, document_id: str):
        # Move the specified Google Docs document to the trash
        try:
            update_body = {'trashed': True}  # Setting the trashed property to True
            self.drive.files().update(fileId=document_id, body=update_body).execute()
            print("Success: Document Moved to Trash")
            #return {"status": "success", "message": "Document moved to trash successfully."}
        except Exception as e:
            print(f"Error: {str(e)}")
            #return {"status": "error", "message": str(e)}

    def delete_docs(self, document_id: str):
        # Delete the specified Google Docs document
        try:
            self.drive.files().delete(fileId=document_id).execute()
            print("Success: Document Deleted")
            #return {"status": "success", "message": "Document deleted successfully."}
        except Exception as e:
            print(f"Error: {str(e)}")
            #return {"status": "error", "message": str(e)}

    def post_video(self, title: str, content: str, tags: set, path: str, publish_time: str):
        # Upload the video file with metadata and scheduled publishing time
        try:
            # Define video metadata
            body = {
                'snippet': {
                    'title': title,
                    'description': content,
                    'categoryId': '24',  # 18: ShortMovies, 42: Shorts, 24: Entertainment,
                    'tags': list({"shorts", "short joke", "funny shorts", "comedy clip", "instant fun", "humor", "funny story", "joke", "english", "study english", "short story", *tags})
                },
                'status': {
                    'privacyStatus': 'public',  # You could choose 'private', 'public' or 'unlisted'
                    'publishAt': publish_time,
                    'selfDeclaredMadeForKids': False
                }
            }

            # Video file to upload
            media = MediaFileUpload(path, mimetype='video/*', resumable=True)

            # Insert video to YouTube
            video = self.youtube.videos().insert(
                part="snippet,status",
                body=body,
                media_body=media
            ).execute()

            # Return the video ID and URL for verification
            return {"status": "success", "videoId": video.get('id')}
        except Exception as e:
            return {"status": "error", "message": str(e)}

    def get_scheduled_videos(self):
        # Retrieve the list of scheduled videos that are scheduled but not yet published
        try:
            # Get the playlists
            response1 = self.youtube.playlistItems().list(
                    part="snippet,contentDetails,status",
                    playlistId=self.playlist_id,
                    maxResults=20,
                    fields='items(snippet(title, resourceId),contentDetails,status)'
                ).execute()
            # Retrieve only videoId whose privacyStatus is 'private' (unpublished)
            unpublished_videoId = [r['snippet']['resourceId']['videoId'] for r in response1['items'] if r['status']['privacyStatus'] == 'private']

            # Get publish time
            if unpublished_videoId:
                response2 = self.youtube.videos().list(
                    part="status",
                    id=','.join(unpublished_videoId)
                ).execute()
                publish_time = [r['status'].get('publishAt', '') for r in response2['items']]

                print(f"Success: {len(publish_time)} scheduled videos are found")
                
            else:
                publish_time = [response1['items'][0]['contentDetails']['videoPublishedAt']]

                print("Success: No scheduled videos are found")
                    
            return publish_time
            #return {"status": "success", "pending_dates": pending_dates}

        except Exception as e:
            
            return {"status": "error", "message": str(e)}

    def generate_future_dates(self, input_times: list, num_dates: int = 10, publish_times: list = ['T12:00:00Z', 'T13:00:00Z'], include_weekends: bool = False):
        # Convert input string times to datetime objects
        dates = [datetime.strptime(time, '%Y-%m-%dT%H:%M:%SZ') for time in input_times]
        # Find the latest date from the inputs
        latest_date = max(dates)
        # List to hold the resulting datetime objects
        result_dates = []
        # Counter to track the number of valid days generated
        count = 0
        # Increment days starting from the day after the latest date
        increment_day = 1

        while count < num_dates:
            # Generate dates for each specified time of publication
            for publish_time in publish_times:
                if count >= num_dates:
                    break
                # Create a new date combining the latest date and the time of publication
                new_date = datetime(latest_date.year, latest_date.month, latest_date.day) + timedelta(days=increment_day)
                new_time = datetime.strptime(publish_time, 'T%H:%M:%SZ').time()
                new_datetime = datetime.combine(new_date, new_time)
                # Check if weekends should be included
                if not include_weekends and new_datetime.weekday() >= 5:  # 5 and 6 are Saturday and Sunday
                    continue
                result_dates.append(new_datetime.isoformat() + 'Z')
                count += 1
            increment_day += 1

        return result_dates


# Create Google API object
google = GoogleAPI()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=260967290589-b75u6164vnmki0764bf4og1hhrrqn27b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50405%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocuments+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=mIK7lFLvLmCyS1eLAhiG0gVKA8w5SK&access_type=offline


In [187]:
schedule = google.get_scheduled_videos()
publish_time = google.generate_future_dates(schedule, num_dates=1)

Success: 2 scheduled videos are found


In [201]:
publish_time

['2024-05-02T12:00:00Z']

In [188]:
content= """📽️ Video Description 📽️: 
Enter a world of robotic humor with Robo and Tinker! 🤖😂 Watch as Tinker tests Robo's buttons, both literally and figuratively, leading to a series of hilarious malfunctions and a playful chase. It's a laugh-out-loud moment of technological mishaps and mischievous fun!

🏫 English Explanation 🗣️: 
1. Robot: A machine capable of carrying out complex actions, often personified in humor.
2. Button: Literally a part of the robot used for operation, and a metaphor for triggering reactions.
3. System overload: A state where a system can't handle the amount of information or operations, used here for comic effect.
4. Anger rising: Typically a human emotion, humorously applied to a robot as part of the pun.
5. Catch me if you can: A playful phrase that turns a technical scenario into a game of tag between Robo and Tinker.

#shorts #LearnEnglish #StudyEnglish #Humor #Funny #Joke #ShortStory #1min #Anglais #AprenderInglés #ingles #英語学習 #英語 #学英语 #英语学习 #영어학습지"""


google.post_video("YouTube Post Test", content, {'test'}, "Videos/1059_CarelessQuery.mp4", publish_time[0])

{'status': 'error',
 'message': '<HttpError 400 when requesting None returned "The request metadata specifies an invalid scheduled publishing time.". Details: "[{\'message\': \'The request metadata specifies an invalid scheduled publishing time.\', \'domain\': \'youtube.video\', \'reason\': \'invalidPublishAt\', \'location\': \'body.status.publishAt\', \'locationType\': \'other\'}]">'}

In [191]:
publish_time[0]

'2024-05-02T12:00:00Z'

In [200]:
body = {
    'snippet': {
        'title': "YouTube Post Test",
        'description': content,
        'categoryId': '24',  # 18: ShortMovies, 42: Shorts, 24: Entertainment,
        'tags': list({"shorts", "short joke", "funny shorts", "comedy clip", "instant fun", "humor", "funny story", "joke", "english", "study english", "short story"})
    },
    'status': {
        'privacyStatus': 'unlisted',  # You could choose 'private', 'public' or 'unlisted'
        'publishAt': '2024-04-26T13:00:00Z',
        'selfDeclaredMadeForKids': False
    }
}

# Video file to upload
media = MediaFileUpload("Videos/1059_CarelessQuery.mp4", mimetype='video/*', resumable=True)

# Insert video to YouTube
video = google.youtube.videos().insert(
    part="snippet,status",
    body=body,
    media_body=media
).execute()


ResumableUploadError: <HttpError 403 when requesting None returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">

In [199]:
publish_time[0]

'2024-05-02T12:00:00Z'

In [ ]:
'2024-04-26T13:00:00Z'

In [194]:
video['status']

{'uploadStatus': 'uploaded',
 'privacyStatus': 'unlisted',
 'license': 'youtube',
 'embeddable': True,
 'publicStatsViewable': True,
 'selfDeclaredMadeForKids': False}

In [195]:
response

{'kind': 'youtube#videoListResponse',
 'etag': 'BC6aok7_ADvIa02ImtXTct7KxG8',
 'items': [{'kind': 'youtube#video',
   'etag': 'LjqV34neFBCBfJ65k0_Tj20V21k',
   'id': 'aC1VbHuGq5A',
   'status': {'uploadStatus': 'processed',
    'privacyStatus': 'private',
    'publishAt': '2024-04-30T04:00:00Z',
    'license': 'youtube',
    'embeddable': True,
    'publicStatsViewable': True,
    'madeForKids': False,
    'selfDeclaredMadeForKids': False}},
  {'kind': 'youtube#video',
   'etag': 'zcr5GrIZ5pdjWwObWjEqZGETc_o',
   'id': 'thuaOSSFWgY',
   'status': {'uploadStatus': 'processed',
    'privacyStatus': 'public',
    'license': 'youtube',
    'embeddable': True,
    'publicStatsViewable': True,
    'madeForKids': False,
    'selfDeclaredMadeForKids': False}}],
 'pageInfo': {'totalResults': 2, 'resultsPerPage': 2}}

In [ ]:
google.youtube.videoCategories().list(
        part='snippet',
        regionCode='US',
    ).execute()

{'kind': 'youtube#videoCategoryListResponse',
 'etag': 'QteLrrS_X7rM7rlcU_e7qa0embQ',
 'items': [{'kind': 'youtube#videoCategory',
   'etag': 'grPOPYEUUZN3ltuDUGEWlrTR90U',
   'id': '1',
   'snippet': {'title': 'Film & Animation',
    'assignable': True,
    'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}},
  {'kind': 'youtube#videoCategory',
   'etag': 'Q0xgUf8BFM8rW3W0R9wNq809xyA',
   'id': '2',
   'snippet': {'title': 'Autos & Vehicles',
    'assignable': True,
    'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}},
  {'kind': 'youtube#videoCategory',
   'etag': 'qnpwjh5QlWM5hrnZCvHisquztC4',
   'id': '10',
   'snippet': {'title': 'Music',
    'assignable': True,
    'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}},
  {'kind': 'youtube#videoCategory',
   'etag': 'HyFIixS5BZaoBdkQdLzPdoXWipg',
   'id': '15',
   'snippet': {'title': 'Pets & Animals',
    'assignable': True,
    'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}},
  {'kind': 'youtube#videoCategory',
   'etag': 'PNU8SwXhjsF90fmkilVohofOi4I',
   'id': '

In [154]:
content= """📽️ Video Description 📽️: 
Enter a world of robotic humor with Robo and Tinker! 🤖😂 Watch as Tinker tests Robo's buttons, both literally and figuratively, leading to a series of hilarious malfunctions and a playful chase. It's a laugh-out-loud moment of technological mishaps and mischievous fun!

🏫 English Explanation 🗣️: 
1. Robot: A machine capable of carrying out complex actions, often personified in humor.
2. Button: Literally a part of the robot used for operation, and a metaphor for triggering reactions.
3. System overload: A state where a system can't handle the amount of information or operations, used here for comic effect.
4. Anger rising: Typically a human emotion, humorously applied to a robot as part of the pun.
5. Catch me if you can: A playful phrase that turns a technical scenario into a game of tag between Robo and Tinker.

#shorts #LearnEnglish #StudyEnglish #Humor #Funny #Joke #ShortStory #1min #Anglais #AprenderInglés #ingles #英語学習 #英語 #学英语 #英语学习 #영어학습지"""


''

### Test

In [2]:
# get list of docs in a specific folder
folder_id = '1FKQa5QuHeDidQMLIlMqad77KLCmr9WOG'
docs = google.list_docs(folder_id)
docs

{'New Document': '1OMVHFiIQWVw2_ry0xnknynu7ykuPqatIgwRNriPoY7I',
 'API Test 02': '1vseDxlFWOAsZrJTlmOX8QrvfsoWNbHlDAk9r7mI6I_E',
 '1059_CarelessQuery': '1OiQVuC5D0AflKGu5beG63NaUsWH1L7LYuzu_TQXmtxA'}

In [5]:
# create a new docs
created_id = google.create_docs("API Test from Python (move_to_trash)")
created_id

Success: New Docs Created


'1EndVlQhjnT8OoGbNTxHn82rquMUgubMxSdrzkeM3flY'

In [7]:
# move docs to trash
#google.move_to_trash(created_id)

Success: Document Moved to Trash


In [4]:
# delete docs
#google.delete_docs(created_id)

Success: Document Deleted


In [8]:
# get document
text_content = google.get_document_content('1OiQVuC5D0AflKGu5beG63NaUsWH1L7LYuzu_TQXmtxA')
text_content

'📽️ Video Description 📽️: \nDive into a tale of digital miscommunication as queries go awry in a comedic exchange of mistaken searches! 🖥️😂 Watch as the confusion unfolds, with each misunderstood prompt leading to more laughs, proving that even in the age of technology, errors can be a source of joy.\n\n🏫 English Explanation 🗣️: \n1. Query: Refers to a request for information, typically in a database or search engine.\n2. Miscommunication: A failure to communicate effectively, which is the source of humor in this scenario.\n3. Digital: Pertaining to technology, emphasizing the modern setting of the joke.\n4. Search: The act of looking for information, leading to funny results from mixed-up inputs.\n5. Prompt: A command or instruction given to a computer, often leading to unexpected outcomes when mistaken.\n\n#shorts #LearnEnglish #StudyEnglish #Humor #Funny #Joke #ShortStory #1min #Anglais #AprenderInglés #ingles #英語学習 #英語 #学英语 #英语学习 #영어학습지\n\n'

In [19]:
print(text_content)

📽️ Video Description 📽️: 
Dive into a tale of digital miscommunication as queries go awry in a comedic exchange of mistaken searches! 🖥️😂 Watch as the confusion unfolds, with each misunderstood prompt leading to more laughs, proving that even in the age of technology, errors can be a source of joy.

🏫 English Explanation 🗣️: 
1. Query: Refers to a request for information, typically in a database or search engine.
2. Miscommunication: A failure to communicate effectively, which is the source of humor in this scenario.
3. Digital: Pertaining to technology, emphasizing the modern setting of the joke.
4. Search: The act of looking for information, leading to funny results from mixed-up inputs.
5. Prompt: A command or instruction given to a computer, often leading to unexpected outcomes when mistaken.

#shorts #LearnEnglish #StudyEnglish #Humor #Funny #Joke #ShortStory #1min #Anglais #AprenderInglés #ingles #英語学習 #英語 #学英语 #英语学习 #영어학습지


